## **PRINCIPLE 1:** Single Responsibility Principle (**SRP**) or Seperation of Concerns (**SOC**)

In [1]:
class Journal:
    def __init__ (self):
        self.entries = []
        self.count = 0
        
    def add_entry (self, text):
        self.count += 1
        self.entries.append(f"{self.count}: {text}")
    
    def remove_entry (self, pos):
        del self.entries[pos]
    def __str__(self):
        return '\n'.join(self.entries) 

In [2]:
j = Journal()
j.add_entry("I ate tomato")
j.add_entry("I slept in time")
print(f"Journal Entries:\n{str(j)}")

Journal Entries:
1: I ate tomato
2: I slept in time


A class should have single responsibility for change, and this should be its main characteristics

## **PRINCIPLE 2:** Open Closed Principle

`OCP - Open for extension, but closed for modification`

In [22]:
from enum import Enum

class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3
    
class Size(Enum):
    SMALL = 1
    MEDIUM = 2
    LARGE = 3
    
class Product:
    def __init__(self, name, color, size):
        self.name = name
        self.color = color
        self.size = size

class ProductFilter: 
    
    def filter_by_color(self, products, color):
        for p in products:
            if p.color == color: yield p
    """
        Suppose later in time we are requested to add the functionality of filtering by size, and someone wrote the
        function below, this is violation of OPC principle, because a class should be closed to modification after 
        production
    """
    def filter_by_size(self, products, size):
        for p in products:
            if p.size == size: yield p
                
# Specification

class Specification:
    
    def is_satisfied(self, item):
        pass
    
    def __and__(self, other):
        return AndSpecification(self, other)
    
class Filter:
    
    def filter(self, items, spec):
        pass
    
# The functions in these classes left unimplemented because we want to extend them by inheriting

# For example let's say we want to have a filter on Color
# The 1st thing we do is to create an extension class for color specification
class ColorSpecification(Specification):
    def __init__(self, color: Color):
        self.color = color
        
    def is_satisfied(self, item):
        return item.color == self.color
    
#Let's say we also want to have a spec class for Size
class SizeSpecification(Specification):
    def __init__(self, size: Size):
        self.size = size
        
    def is_satisfied(self, item):
        return item.size == self.size

class AndSpecification(Specification):
    def __init__(self, *args):
        self.args = args
        
    def is_satisfied(self, item):
        return all(map(
            lambda spec: spec.is_satisfied(item), self.args
        ))


class BetterFilter(Filter):
    def filter(self, items, spec: Specification):
        for item in items:
            if spec.is_satisfied(item): yield item

In [24]:
apple = Product("apple", Color.RED, Size.SMALL)
car   = Product("car", Color.BLUE, Size.MEDIUM)
house = Product("house", Color.GREEN, Size.LARGE)

green_color_spec = ColorSpecification(Color.GREEN)
large_size_spec  = SizeSpecification(Size.LARGE)
green_and_large_spec = green_color_spec & large_size_spec

bf = BetterFilter()

products = [apple, car, house]
for fp in bf.filter(products, green_and_large_spec):
    print(f"{fp.name} - is large and green")

house - is large and green


## Principle 3: Liskov Substitution Principle (LSP)

`LSP - If there is an interface based on a base class, you should stick a drived class in there and everything should work`

In [3]:
class Rectangle:
    def __init__(self, width, height):
        self._width = width
        self._height = height
    
    def __str__(self):
        return f"Width: {self.width}, height: {self.height}"
    
    @property
    def area(self):
        return self._width * self._height
    
    @property
    def width(self):
        return self._width
    
    @property
    def height(self):
        return self._height
    
    @width.setter
    def width(self, value):
        self._width = width
    
    @height.setter
    def height(self, value):
        self._height = value

In [7]:
class Square(Rectangle):
    def __init__(self, size):
        Rectangle.__init__(self, size, size)
    
    @Rectangle.width.setter
    def width(self, value):
        self._width = self._height = value
        
    @Rectangle.height.setter
    def height(self, value):
        self._height = self._width = value

In [4]:
def use_it(rc: Rectangle):
    w = rc.width
    rc.height = 10
    
    expected = int(w*10)
    
    print(f"Expected an area of {expected}, but got {rc.area}")

In [5]:
rc = Rectangle(2,3)
use_it(rc)

Expected an area of 20, but got 20


In [8]:
sq = Square(5)
use_it(sq)

Expected an area of 50, but got 100


This is violation of Liskov's substitution principle, because if we have an interface working with base class, then we should be able to stick in any derived inheritors, and the interface should work just fine. **But in this case in Square class (which inherits the Rectangle class) the function broke**.

  ## Principle 4: Interface Segregation Principle (ISP)

`ISP` implies if a class is designed to be an interface, it is not ideal to implement many methods in single interface that can be seperated. Instead the related methods could also be implemented in a seperate interface, and the current interface could inherit from it

## Principle 5: Dependency Inversion Principle (DIP)

`DIP` implies that the high level classes (modules) should not be directly dependent on the low level classes, but instead on the abstractions of low level classes

In [24]:
from enum import Enum
from abc import abstractmethod, ABC, ABCMeta

In [6]:
class Relationship(Enum):
    PARENT = 0
    CHILD = 1
    SIBLING = 2
    
class Person:
    def __init__(self, name):
        self.name = name
        
class Relationships:
    def __init__(self):
        self.relations = []
        
    def add_parent_and_child(self, parent: Person, child: Person):
        self.relations.append(
            (parent, Relationship.PARENT, child)
        )
        
        self.relations.append(
            (child, Relationship.CHILD, parent)
        )
        
# Suppose someone now decides to create a Resaerch class that searches if there is parent called "John" 
# and wether he has a child in relationship tree

class Research:
    def __init__(self, relationships: Relationships):
        relations = relationships.relations
        for r in relations: 
            if (r[0].name == "John" and r[1] == Relationship.PARENT):
                print(f"John has a child called {r[2].name}")

parent = Person("John")
child1 = Person("Chris")
child2 = Person("Matt")

r = Relationships()
r.add_parent_and_child(parent, child1)
r.add_parent_and_child(parent, child2)

Research(r) 

# While this works, there s a problem. This problem is Research class has dependency on relations being a 
# list (iterable), thats why if someday the relations prop of low level class Relationships changes the high level class
# Research will fail

John has a child called Chris
John has a child called Matt


In [36]:
# One idea is to provide a utility methods for performing search ops right in low level class

# Initially we define a class called RelationshipBrowswer

class RelationshipBrowser(ABC):
    @abstractmethod
    def find_all_children_of(self, name): pass
        
class Relationships(RelationshipBrowser):
    def __init__(self):
        self.relations = []
        
    def add_parent_and_child(self, parent: Person, child: Person):
        self.relations.append(
            (parent, Relationship.PARENT, child)
        )
        self.relations.append(
            (child, Relationship.CHILD, parent)
        )
    
    def find_all_children_of(self, name):
        # implement method accordingly 
        
        for r in self.relations:
            if (r[0].name == name and r[1] == Relationship.PARENT):
                yield r[2].name
            
class Research:
    def __init__(self, browser: RelationshipBrowser):
        for child_name in browser.find_all_children_of("John"):
            print(f"John has a child named {child_name}")

In [37]:
parent = Person("John")
child1 = Person("Chris")
child2 = Person("Matt")

r = Relationships()
r.add_parent_and_child(parent, child1)
r.add_parent_and_child(parent, child2)

Research(r)

John has a child named Chris
John has a child named Matt
